In [1]:
import sqlite3 as sql
from contextlib import closing
import pandas as pd
import numpy as np
import time as tm
import pickle
import os
import csv

from index import my_index
from execute import my_execute

sqlite3 does not support contextual closing natively (yet). Using a super-elegant workaround proposed by erlendaasland\
https://discuss.python.org/t/implicitly-close-sqlite3-connections-with-context-managers/33320/3

In [2]:
def safe_tran( db_name, query ):
  with closing( sql.connect( db_name ) ) as conn:
    cur = conn.execute( query )
    cols = [ col[0] for col in cur.description ]
    df = pd.DataFrame.from_records( cur, columns = cols )
    return df

db_name = "public.db"
get_gold_results = lambda query: safe_tran( db_name, query )

In [3]:
def make_sqlite_query( clause ):
  query = "SELECT id FROM tbl WHERE "
  query += " AND ".join( [ ' '.join( pred ) for pred in clause ] )
  return query

def eval_results( clause, disk, idx_stat ):
  # Execute the query on an actual DB
  df_gold = get_gold_results( make_sqlite_query( clause ) )

  # Execute the query using the index and time it
  tic = tm.perf_counter()
  diskloc_list = my_execute( clause, idx_stat )
  # print( diskloc_list )
  toc = tm.perf_counter()
  t_idx = toc - tic

  # print(type(diskloc_list[0]))

  # Do sanity checks on the returned locations -- dont want any buffer overflow attacks :)
  diskloc_list = np.minimum( np.maximum( diskloc_list, 0 ), len( disk ) - 1 )

  # Find the seek and read time requried to retrieve records from the virtual disk
  diffs = diskloc_list[ 1: ] - diskloc_list[ :-1 ]
  # Take care of cases where we need to loop back to reach a record
  diffs[ diffs <= 0 ] += len( disk )
  t_seek = diffs.sum()
  t_read = len( diskloc_list )
  # Sanity check
  assert( t_seek >= t_read - 1 )
  t_seek -= t_read - 1
  # Take care of pesky edge cases
  if t_read == 0:
    t_seek = 0

  # Get hold of the tuples chosen by the index from the virtual disk
  response_stu = []
  if len( diskloc_list ) > 0:
    response_stu = disk[ diskloc_list ]
  df_stu = pd.DataFrame( response_stu, columns = [ "id" ] )

  # Rename columns just to be safe so as to enable merging
  df_stu.rename( dict( zip( df_stu.columns, df_gold.columns ) ), axis = 1, inplace = True )

  union = pd.merge( df_gold, df_stu, how = "outer", indicator = True )
  inter = pd.merge( df_gold, df_stu, how = "inner", indicator = True )


  print(len( inter ), len( df_gold ), len(df_stu))
  # print("gold list: ", df_gold)
  # print("student list: ", df_stu)
  # If the gold response is not empty, use intersection over union score
  # Since union removes duplicates, consider length of diskloc_list as well
  if len( df_gold ) > 0:
    score = round( len( inter ) / max( len( diskloc_list ), len( union ) ), 2 )
  # If the gold response itself is empty, penalize non-empty response by index
  elif len( df_gold ) == 0:
    score = round( 1 / ( 1 + len( diskloc_list ) ), 2 )

  return t_idx, t_seek, t_read, score

In [4]:
n_trials = 3

t_build = 0
disk_size = np.int64(0)
idx_size = 0
t_idx = 0
t_seek = np.int64(0)
t_read = np.int64(0)
score = 0

In [5]:
# Read the data to be indexed
with open( "public.csv", 'r' ) as csvfile:
  reader = csv.reader( csvfile )
  tuples = [ ( int( row[ 0 ] ), row[ 1 ], int( row[ 2 ] ) ) for row in reader ]

# Create proper predicates out of CSV data
def make_predicates( tok_list ):
  if len( tok_list ) == 3:
    return [ tok_list ]
  if len( tok_list ) == 6:
    return [ tok_list[ :3 ], tok_list[ 3: ] ]

# Read the clauses that will constitute the evaluation queries
with open( "clauses.csv", 'r' ) as csvfile:
  reader = csv.reader( csvfile )
  c_list = [ make_predicates( row ) for row in reader ]

In [6]:
for t in range( n_trials ):
  tic = tm.perf_counter()
  disk, idx_stat = my_index( tuples )
  disk = np.array( disk )
  toc = tm.perf_counter()
  t_build += toc - tic

  disk_size += len( disk )

  with open( f"idx_dump_{t}.pkl", "wb" ) as outfile:
    pickle.dump( idx_stat, outfile, protocol=pickle.HIGHEST_PROTOCOL )

  idx_size += os.path.getsize( f"idx_dump_{t}.pkl" )

  c_list_len = 0
  for clause in c_list:
    # if len( clause ) == 1:
    #   continue
    # if clause[0][0] == 'name':
    #   continue
    # if clause[0][0] == 'year':
    #   continue
    print( clause )
    c_list_len = c_list_len + 1
    t_i, t_s, t_r, scr = eval_results( clause, disk, idx_stat )
    t_idx += t_i
    t_seek += t_s
    t_read += t_r
    score += scr

[['year', '>=', '1969']]
99786 99786 99786
[['name', 'LIKE', "'je%'"], ['year', '>=', '1955']]
4948 4948 4948
[['year', '>=', '2011']]
42496 42496 42496
[['year', '<=', '2030']]
100000 100000 100000
[['year', '<=', '1997']]
29353 29353 29353
[['name', 'LIKE', "'s%'"], ['year', '<=', '1947']]
0 0 0
[['name', 'LIKE', "'so%'"], ['year', '>=', '1996']]
2018 2018 2018
[['name', 'LIKE', "'j%'"], ['year', '>=', '2019']]
5513 5513 5513
[['name', '=', "'fajaj'"]]
40 40 40
[['name', 'LIKE', "'j%'"], ['year', '>=', '1999']]
18202 18202 18202
[['name', 'LIKE', "'fese%'"]]
53 53 53
[['name', '=', "'falis'"], ['year', '>=', '1962']]
5 5 5
[['name', 'LIKE', "'jej%'"]]
931 931 931
[['name', 'LIKE', "'j%'"], ['year', '>=', '2017']]
7465 7465 7465
[['name', '=', "'fejis'"]]
7 7 7
[['name', '=', "'jijic'"], ['year', '<=', '2016']]
3 3 3
[['name', 'LIKE', "'jo%'"], ['year', '<=', '1947']]
0 0 0
[['name', 'LIKE', "'jab%'"], ['year', '>=', '2039']]
0 0 0
[['name', 'LIKE', "'s%'"], ['year', '<=', '2036']]
16

In [7]:
t_build /= n_trials
disk_size /= n_trials
idx_size /= n_trials
t_idx /= n_trials
t_seek /= n_trials
t_read /= n_trials
score /= n_trials
# score /= len( c_list )
score /= c_list_len

idx_size /= 10000000
t_seek /= 1000000
t_read /= 1000000

print( t_build, disk_size, idx_size, t_idx, t_seek, t_read, score )

with open("results.csv", "a", newline = "") as file:
    # Write the output to the file
    model = " ------ (year, name, joint) = (year, name, trie-2)"
    writer = csv.writer(file)
    writer.writerow([t_build, disk_size, idx_size, t_idx, t_seek, t_read, score, model])

3.6992956253333342 300000.0 1.3336235 0.10278811366667018 2.22776 1.400713 1.0
